This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict
from math import log, floor, ceil

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
#from mastml.utils import nice_range
#from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

import forestci as fci

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

#logger = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value, nice_names):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value, nice_names=nice_names())
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


def get_histogram_bins(y_df):
    """
    Method to obtain the number of bins to use when plotting a histogram

    Args:

        y_df: (pandas Series or numpy array), array of y data used to construct histogram

    Returns:

        num_bins: (int), the number of bins to use when plotting a histogram

    """

    bin_dividers = np.linspace(y_df.shape[0], 0.05*y_df.shape[0], y_df.shape[0])
    bin_list = list()
    try:
        for divider in bin_dividers:
            if divider == 0:
                continue
            bins = int((y_df.shape[0])/divider)
            if bins < y_df.shape[0]/2:
                bin_list.append(bins)
    except:
        num_bins = 10
    if len(bin_list) > 0:
        num_bins = max(bin_list)
    else:
        num_bins = 10
    return num_bins


def nice_names():
    nice_names = {
    # classification:
    'accuracy': 'Accuracy',
    'f1_binary': '$F_1$',
    'f1_macro': 'f1_macro',
    'f1_micro': 'f1_micro',
    'f1_samples': 'f1_samples',
    'f1_weighted': 'f1_weighted',
    'log_loss': 'log_loss',
    'precision_binary': 'Precision',
    'precision_macro': 'prec_macro',
    'precision_micro': 'prec_micro',
    'precision_samples': 'prec_samples',
    'precision_weighted': 'prec_weighted',
    'recall_binary': 'Recall',
    'recall_macro': 'rcl_macro',
    'recall_micro': 'rcl_micro',
    'recall_samples': 'rcl_samples',
    'recall_weighted': 'rcl_weighted',
    'roc_auc': 'ROC_AUC',
    # regression:
    'explained_variance': 'expl_var',
    'mean_absolute_error': 'MAE',
    'mean_squared_error': 'MSE',
    'mean_squared_log_error': 'MSLE',
    'median_absolute_error': 'MedAE',
    'root_mean_squared_error': 'RMSE',
    'rmse_over_stdev': r'RMSE/$\sigma_y$',
    'R2': '$R^2$',
    'R2_noint': '$R^2_{noint}$',
    'R2_adjusted': '$R^2_{adjusted}$',
    'R2_fitted': '$R^2_{fitted}$'
    }
    return nice_names


def nice_range(lower, upper):
    """
    Method to create a range of values, including the specified start and end points, with nicely spaced intervals

    Args:

        lower: (float or int), lower bound of range to create

        upper: (float or int), upper bound of range to create

    Returns:

        (list), list of numerical values in established range

    """

    flipped = 1 # set to -1 for inverted

    # Case for validation where nan is passed in
    if np.isnan(lower):
        lower = 0
    if np.isnan(upper):
        upper = 0.1

    if upper < lower:
        upper, lower = lower, upper
        flipped = -1
    return [_int_if_int(x) for x in _nice_range_helper(lower, upper)][::flipped]


def nice_mean(ls):
    """
    Method to return mean of a list or equivalent array with NaN values

    Args:

        ls: (list), list of values

    Returns:

        (numpy array), array containing mean of list of values or NaN if list has no values

    """

    if len(ls) > 0:
        return np.mean(ls)
    return np.nan


def nice_std(ls):
    """
    Method to return standard deviation of a list or equivalent array with NaN values

    Args:

        ls: (list), list of values

    Returns:

        (numpy array), array containing standard deviation of list of values or NaN if list has no values

    """
    if len(ls) > 0:
        return np.std(ls)
    return np.nan


def rounder(delta):
    """
    Method to obtain number of decimal places to report on plots

    Args:

        delta: (float), a float representing the change in two y values on a plot, used to obtain the plot axis spacing size

    Return:

        (int), an integer denoting the number of decimal places to use

    """
    if 0.001 <= delta < 0.01:
        return 3
    elif 0.01 <= delta < 0.1:
        return 2
    elif 0.1 <= delta < 1:
        return 1
    elif 1 <= delta < 100000:
        return 0
    else:
        return 0


def _set_tick_labels(ax, maxx, minn):
    """
    Method that sets the x and y ticks to be in the same range

    Args:

        ax: (matplotlib axes object), a matplotlib axes object

        maxx: (float), a maximum value

        minn: (float), a minimum value

    Returns:

        None

    """
    _set_tick_labels_different(ax, maxx, minn, maxx, minn) # I love it when this happens


def _set_tick_labels_different(ax, max_tick_x, min_tick_x, max_tick_y, min_tick_y):
    """
    Method that sets the x and y ticks, when the axes have different ranges

    Args:

        ax: (matplotlib axes object), a matplotlib axes object

        max_tick_x: (float), the maximum tick value for the x axis

        min_tick_x: (float), the minimum tick value for the x axis

        max_tick_y: (float), the maximum tick value for the y axis

        min_tick_y: (float), the minimum tick value for the y axis

    Returns:

        None

    """

    tickvals_x = nice_range(min_tick_x, max_tick_x)
    tickvals_y = nice_range(min_tick_y, max_tick_y)

    if tickvals_x[-1]-tickvals_x[len(tickvals_x)-2] < tickvals_x[len(tickvals_x)-3]-tickvals_x[len(tickvals_x)-4]:
        tickvals_x = tickvals_x[:-1]
    if tickvals_y[-1]-tickvals_y[len(tickvals_y)-2] < tickvals_y[len(tickvals_y)-3]-tickvals_y[len(tickvals_y)-4]:
        tickvals_y = tickvals_y[:-1]
    #tickvals_x = _clean_tick_labels(tickvals=tickvals_x, delta=max_tick_x-min_tick_x)
    #tickvals_y = _clean_tick_labels(tickvals=tickvals_y, delta=max_tick_y - min_tick_y)

    ax.set_xticks(ticks=tickvals_x)
    ax.set_yticks(ticks=tickvals_y)

    ticklabels_x = [str(tick) for tick in tickvals_x]
    ticklabels_y = [str(tick) for tick in tickvals_y]

    rotation = 0
    # Look at length of x tick labels to see if may be possibly crowded. If so, rotate labels
    tick_length = len(str(tickvals_x[1]))
    if tick_length >= 4:
        rotation = 45
    ax.set_xticklabels(labels=ticklabels_x, fontsize=14, rotation=rotation)
    ax.set_yticklabels(labels=ticklabels_y, fontsize=14)


def _nice_range_helper(lower, upper):
    """
    Method to help make a better range of axis ticks

    Args:

        lower: (float), lower value of axis ticks

        upper: (float), upper value of axis ticks

    Returns:

        upper: (float), modified upper tick value fixed based on set of axis ticks

    """
    steps = 8
    diff = abs(lower - upper)

    # special case where lower and upper are the same
    if diff == 0:
        return [lower,]

    # the exact step needed
    step = diff / steps

    # a rough estimate of best step
    step = _nearest_pow_ten(step) # whole decimal increments

    # tune in one the best step size
    factors = [0.1, 0.2, 0.5, 1, 2, 5, 10]

    # use this to minimize how far we are from ideal step size
    def best_one(steps_factor):
        steps_count, factor = steps_factor
        return abs(steps_count - steps)
    n_steps, best_factor = min([(diff / (step * f), f) for f in factors], key = best_one)

    #print('should see n steps', ceil(n_steps + 2))
    # multiply in the optimal factor for getting as close to ten steps as we can
    step = step * best_factor

    # make the bounds look nice
    lower = _three_sigfigs(lower)
    upper = _three_sigfigs(upper)

    start = _round_up(lower, step)

    # prepare for iteration
    x = start # pointless init
    i = 0

    # itereate until we reach upper
    while x < upper - step:
        x = start + i * step
        yield _three_sigfigs(x) # using sigfigs because of floating point error
        i += 1

    # finish off with ending bound
    yield upper


def _nearest_pow_ten(x):
    """
    Method to return the nearest power of ten for an axis tick value

    Args:

        x: (float), an axis tick number

    Returns:

        (float), nearest power of ten of x

    """
    sign = 1
    if x == 0:
        return 0
    if x < 0: # case for negatives
        x = -x
        sign = -1
    return sign*10**ceil(log(x, 10))


def _three_sigfigs(x):
    """
    Method invoking special case of _n_sigfigs to return 3 sig figs

    Args:

        x: (float), an axis tick number

    Returns:

        (float), number of sig figs (always 3)

    """
    return _n_sigfigs(x, 3)


def _n_sigfigs(x, n):
    """
    Method to return number of sig figs to use for axis ticks

    Args:

        x: (float), an axis tick number

    Returns:

        (float), number of sig figs

    """
    sign = 1
    if x == 0:
        return 0
    if x < 0: # case for negatives
        x = -x
        sign = -1
    if x < 1:
        base = n - round(log(x, 10))
    else:
        base = (n-1) - round(log(x, 10))
    return sign * round(x, base)


def _int_if_int(x):
    """
    Method to return integer mapped value of x

    Args:

        x: (float or int), a number

    Returns:

        x: (float), value of x mapped as integer

    """
    if int(x) == x:
        return int(x)
    return x


def _round_up(x, inc):
    """
    Method to round up the value of x

    Args:

        x: (float or int), a number

        inc: (float), an increment for axis ticks

    Returns:

        (float), value of x rounded up

    """
    sign = 1
    if x < 0: # case for negative
        x = -x
        sign = -1

    return sign * inc * ceil(x / inc)


def prediction_intervals(model, X, rf_error_method, rf_error_percentile, Xtrain, Xtest):
    """
    Method to calculate prediction intervals when using Random Forest and Gaussian Process regression models.

    Prediction intervals for random forest adapted from https://blog.datadive.net/prediction-intervals-for-random-forests/

    Args:

        model: (scikit-learn model/estimator object), a scikit-learn model object

        X: (numpy array), array of X features

        method: (str), type of error bar to formulate (e.g. "stdev" is standard deviation of predicted errors, "confint"
        is error bar as confidence interval

        percentile: (int), percentile for which to form error bars

    Returns:

        err_up: (list), list of upper bounds of error bars for each data point

        err_down: (list), list of lower bounds of error bars for each data point

    """
    err_down = list()
    err_up = list()
    X_aslist = X.values.tolist()
    if model.__class__.__name__ in ['RandomForestRegressor', 'GradientBoostingRegressor']:

        #if rf_error_method == 'jackknife':
        #    #new method based on http://contrib.scikit-learn.org/forest-confidence-interval/auto_examples/plot_mpg.html#sphx-glr-auto-examples-plot-mpg-py
        #    #inbag = fci.calc_inbag(Xtrain.shape[0], model)
        #    random_forest_variances = fci.random_forest_error(model, X_train=Xtrain, X_test=Xtest, calibrate=False)
        #    # remove NaN values from array
        #    random_forest_stdevs = np.sqrt(random_forest_variances)
        #    indices_to_ignore = np.argwhere(np.isnan((random_forest_stdevs)))
        #    random_forest_stdevs = random_forest_stdevs[~np.isnan(random_forest_stdevs)]
        #    err_up = random_forest_stdevs
        #    err_down = random_forest_stdevs
        for x in range(len(X_aslist)):
            preds = list()
            if model.__class__.__name__ == 'RandomForestRegressor':
                for pred in model.estimators_:
                    preds.append(pred.predict(np.array(X_aslist[x]).reshape(1,-1))[0])
            elif model.__class__.__name__ == 'GradientBoostingRegressor':
                for pred in model.estimators_.tolist():
                    preds.append(pred[0].predict(np.array(X_aslist[x]).reshape(1,-1))[0])
            if rf_error_method == 'confint':
                e_down = np.percentile(preds, (100 - int(rf_error_percentile)) / 2.)
                e_up = np.percentile(preds, 100 - (100 - int(rf_error_percentile)) / 2.)
            elif rf_error_method == 'stdev':
                e_down = np.std(preds)
                e_up = np.std(preds)
            elif rf_error_method == 'False' or rf_error_method is False:
                # basically default to stdev
                e_down = np.std(preds)
                e_up = np.std(preds)
            if e_up == 0.0:
                e_up = 10 ** 10
            if e_down == 0.0:
                e_down = 10 ** 10
            err_down.append(e_down)
            err_up.append(e_up)

    if model.__class__.__name__=='GaussianProcessRegressor':
        preds = model.predict(X, return_std=True)[1] # Get the stdev model error from the predictions of GPR
        err_up = preds
        err_down = preds

    #if model.__class__.__name__=='ModelImport':
    #    if model.model.__class__.__name__=='GaussianProcessRegressor':
    #        from sklearn.gaussian_process import GaussianProcessRegressor
    #        kernel = model.model.kernel_
    #        gprmodel = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
    #        preds = gprmodel.predict(X, return_std=True)[1] # Get the stdev model error from the predictions of GPR
    #        err_up = preds
    #        err_down = preds
    return err_down, err_up


In [ ]:
def plot_target_histogram(y_df, savepath, title='target histogram', label='target values'):
    """
    Method to plot the histogram of true y values

    Args:

        y_df: (pandas dataframe), dataframe of true y data values

        savepath: (str), path to save the plotted precision-recall curve

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.70
    fig, ax = make_fig_ax(aspect_ratio=0.5, x_align=x_align)

    #ax.set_title(title)

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=y_df)

    # do the actual plotting
    try:
        ax.hist(y_df, bins=num_bins, color='b', edgecolor='k')#, histtype='stepfilled')
    except:
        print('Could not plot target histgram')
        return
    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))


    plot_stats(fig, dict(y_df.describe()), x_align=x_align, y_align=0.90, fontsize=14)
    # Save input data stats to csv
    savepath_parse = savepath.split('target_histogram.png')[0]
    y_df.describe().to_csv(savepath_parse+'/''input_data_statistics.csv')

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
from sklearn.gaussian_process import GaussianProcessRegressor  # Need for error plots
from sklearn.gaussian_process.kernels import *  # Need for error plots
from sklearn.ensemble import RandomForestRegressor  # Need for error plots
y_df = pd.Series(pd.read_csv(StringIO('''
13.105
9.33
7.95
1.0
6.0
0.7
10.48
4.17
3.61
0.67
2.89
0.93
8.0
7.0
2.0
1.46
2.6
6.244999999999999
3.7
2.26
10.5
2.4
8.97
2.0
7.5
1.1
2.3
5.89
7.77
11.8
1.435
1.4
1.086
2.55
5.6
0.1575
0.82
4.7
0.040999999999999995
0.1
0.82
5.74
9.7
2.45
4.1
2.5
3.1
1.919
2.5
0.4
2.53
5.0
11.0
1.12
1.89
1.45
1.72
0.59
1.98
2.015
0.965
0.23
3.82
10.6
2.1
8.5
1.8
7.6
1.3999999999999997
1.9
6.17
4.5
7.7
10.0
1.9
6.0
4.87
0.9
4.07
5.98
0.46
2.6
6.0
2.0
0.3
0.04
3.15
1.24
0.3
0.9
0.69
1.0
0.3
0.1
2.34
0.65
0.66
0.15
0.8
1.68
0.23
0.35
0.9
0.015
0.025
0.2
0.02
0.26
3.7
10.05
0.58
0.6
6.2
0.2
1.8
1.25
0.48
4.04
2.34
0.9
0.1
0.4
1.2
0.2
0.77
0.17
0.6
0.46
5.15
0.47
0.045
0.15
0.25
0.2
3.7
0.5
0.12
0.5
0.05
0.23
4.17
2.023
1.4
1.21
3.306
1.23
2.987
1.08
3.07
3.35
1.9275
1.15
3.87
0.9
0.93
2.67
0.745
0.22
1.2
2.25
4.53
3.24
4.54
2.5
2.5
3.59
1.42
2.05
2.05
0.725
1.42
1.225
0.1
5.56
1.58
3.54
0.335
1.05
1.225
1.315
2.4325
0.8
0.84
0.3
1.5
4.0
2.43
2.4
2.48
1.2
1.215
1.77
2.6
0.48
0.74
2.29
1.9
1.7266666666666666
0.64
1.87
10.4
8.29
7.2
1.0
4.5
5.83
5.77
4.76
4.42
3.73
1.9
5.6
0.6
0.18
0.2
4.99
2.215
2.17
2.0
1.25
0.25
3.48
0.12
1.4
3.3083333333333336
1.0699999999999998
0.83
1.0
1.8
0.8
1.0
0.3
0.25
0.8
1.5
0.6
0.7
1.5
0.65
0.5
0.75
0.2
0.4
1.8
3.9
1.2
0.95
4.19
0.13
3.4850000000000003
0.064
0.85
1.75
6.0
1.785
0.52
1.0
2.4175
5.7
1.0
0.13
1.714
4.47
0.6
0.45
1.25
1.517
3.5615
0.2
2.4
3.1845
1.351
1.565
1.86
0.9
0.35600000000000004
1.24
0.7390000000000001
1.2
0.17
0.98
3.5
2.475
1.08
3.9
1.5
1.055
3.4
0.075
0.18
2.38
2.4
3.31
1.9
1.7
0.009000000000000001
1.2
0.1
0.3
2.22
3.0
0.332
1.1
1.3
10.0
8.1
0.6
7.18
1.6
3.5
6.37
0.55
0.55
5.13
9.06
3.88
3.6
3.4
0.08
1.05
6.6
1.32
1.97
0.8
0.1
0.43
0.7
2.68
4.85
1.12
1.75
1.4
4.9
0.66
0.84
5.9
1.0
1.8
0.66
1.1540000000000001
1.3
4.9
0.97
6.15
3.0
1.04
1.7
0.29
0.3
0.38
1.12
4.9
1.17
1.09
0.22
3.0
1.03
0.46
2.3
0.59
0.62
1.25
1.122
0.6
1.7
1.2
1.645
0.163
0.6
1.78
1.959
3.6
1.54
1.36
0.63
1.3
0.5
0.95
0.4
3.3
2.6
3.08
2.91
1.0
1.79
2.84
3.5
2.4
3.26
1.07
1.66
0.9
0.9
0.9
1.18
2.99
0.33
1.02
1.0
3.94
5.55
2.43
2.8
1.665
2.3
4.6
0.1
2.8
1.1
1.35
0.05
2.3
0.12
1.1
0.99
1.2
2.0
0.9
0.2
0.3
0.9
1.0
0.2
0.8
0.8
0.75
0.55
0.1
0.11
2.48
1.269
3.5
0.24
2.6
3.59
0.175
2.13
3.58
1.825
1.36
3.56
0.73
3.05
0.7
0.5
2.67
3.02
2.512
1.67
1.85
2.12
0.41
3.82
0.27
3.2
0.31
2.32
2.755
1.2
0.21
2.66
0.145
1.9625
0.015
4.625
1.5
2.18
''')).iloc[:,0])
savepath = '/home/nanohub/rjiang/intromllab/bin/./generated_features_0413/target_histogram.png'
label = 'Band gap values Clean'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_target_histogram(y_df, savepath, label)
display(Image(filename='target_histogram.png'))
